
## Registing Google Drive and Google Earth Engine (GEE) account.
#### Attention: Using edu email account may cannot creat Google Earth Engine project, but may have unlimitted goodle drive storage capacity.
#### Tip: Using personal email account to register GEE account, creat GEE project. Using edu account to register both Google drive and GEE account. Share the created GEE project with the edu GEE account.  


# Code for time serious rainfall data download

### Using edu GEE account to deal with the shared GEE project.
#### 1. Login Google Cloud, go to Google Cloud Dashboard, --> APIs & Services --> Enabled APIs and Services --> enable "Google earth engine Service".
#### 2. Google Cloud Dashboard --> IAM & Admin --> Service account --> Creat Service account --> Keys --> Add key (generate and download key in json format). The service account and the key will be used in the code.
#### 3 In the edu Google Drive, creat a folder for this project, and share this foler with the service account generated in the above step. 

In [ ]:
import ee
import geopandas as gpd
import os
import pandas as pd
from datetime import datetime, timedelta
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Earth Engine service account configuration
SERVICE_ACCOUNT = 'service_account@windy-winter-456502-b1.iam.gserviceaccount.com'
KEY_FILE = '/downloaded_files/windy-winter-456502-b1-d9e863d374df.json'

In [ ]:
credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY_FILE)
ee.Initialize(credentials)

# Input shapefile and output folder
input_shapefile = "/downloaded_files/Flooding_height_points.shp"
# The folder created in edu Google drive account, and shared with the created service account.
output_drive_folder = "Flooding_CHIRPS"

# CHIRPS configuration
CHIRPS = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
CHIRPS_BAND = "precipitation"
CHIRPS_SCALE = 5566
IMG_SIZE = 300

# GEE task control parameters
MAX_ACTIVE_TASKS = 3000  # Maximum number of concurrent tasks
CHECK_INTERVAL = 30  # Check interval in seconds

def get_active_task_count():
    """Get the current number of active GEE tasks (READY or RUNNING)"""
    return sum(1 for task in ee.batch.Task.list() if task.state in ['READY', 'RUNNING'])

def wait_for_available_task_slot(max_active=MAX_ACTIVE_TASKS):
    """Wait until the current number of tasks is below the maximum allowed"""
    while True:
        active_count = get_active_task_count()
        if active_count < max_active:
            break
        print(f"⏳ Current number of active tasks: {active_count}, waiting...")
        time.sleep(CHECK_INTERVAL)

def export_chirps_image(point, date_str, out_name, scale=CHIRPS_SCALE, folder=output_drive_folder, size_px=300):
    try:
        start = ee.Date(date_str)
        end = start.advance(1, 'day')
        image = CHIRPS.filterDate(start, end).first().select(CHIRPS_BAND)

        # Check if the image exists
        try:
            if image.get('system:id').getInfo() is None:
                print(f"❌ {out_name}: No image found, skipping")
                return
        except:
            print(f"❌ {out_name}: Failed to get image, skipping")
            return

        region = point.buffer(scale * size_px / 2).bounds()

        wait_for_available_task_slot()  # Check task count

        task = ee.batch.Export.image.toDrive(
            image=image,
            description=out_name,
            folder=folder,
            fileNamePrefix=out_name,
            region=region.getInfo()['coordinates'],
            crs='EPSG:4326',
            maxPixels=1e9,
            fileFormat='GeoTIFF',
            dimensions=[size_px, size_px]
        )
        task.start()
        print(f"✅ Export task submitted: {out_name}")

    except Exception as e:
        print(f"❌ {out_name}: Export failed: {e}")

def main():
    gdf = gpd.read_file(input_shapefile)
    print(f"📄 Loaded {len(gdf)} points")

    tasks = []
    with ThreadPoolExecutor(max_workers=20) as executor:  # 20 can be adjusted as needed
        for idx, row in gdf.iterrows():
            point_id = str(row['ID'])
            lat = row['latitude_d']
            lon = row['longitude_']
            peak_date = row.get('peak_date')

            if pd.isnull(peak_date) or str(peak_date).lower() in ['none', 'nan']:
                print(f"⚠️ ID {point_id} has invalid peak_date, skipping")
                continue

            try:
                point = ee.Geometry.Point(float(lon), float(lat))
                peak_dt = pd.to_datetime(peak_date).date()
            except Exception as e:
                print(f"⚠️ Coordinate or date parsing error for ID {point_id}: {e}")
                continue
#        for i in range(1, 7):  # Previous 6 days
#            target_date = peak_dt - timedelta(days=i)
#            date_str = target_date.strftime('%Y-%m-%d')
#            out_name = f"{point_id}_{date_str}_CHIRPS"
#            export_chirps_image(point, date_str, out_name)

            date_str = peak_dt.strftime('%Y-%m-%d')
            out_name = f"{point_id}_{date_str}_CHIRPS"
            # Submit with thread pool concurrently
            tasks.append(executor.submit(
                export_chirps_image, point, date_str, out_name
            ))

 

    print("🎯 All export tasks have been submitted. Please check progress on the GEE Tasks page.")

if __name__ == "__main__":
    main()
